## SMTP发送邮件
Python 对SMTP支持有smtplib和email两个模块，email负责构造邮件，smtplib复制发送邮件。
1. 构造一个最简单的纯文本邮件：
  - 构造MIMEText对象时，第一个参数就是邮件正文，第二个参数是MIME的subtype，传入'plain'表示纯文本。
  - 最终的MIME就是'text/plain'，最后一定要用utf-8编码保证多语言兼容性。
  - set_debuglevel(1)就可以打印出和SMTP服务器交互的所有信息。SMTP协议就是简单的文本命令和响应。login()方法用来登录SMTP服务器，sendmail()方法就是发送邮件， 由于可以一次发送个多个人，所以传入一个list，邮件正文是一个str，as_string()把MIMEText对象变成str。

In [1]:
from email.mime.text import MIMEText
from email.header import Header
from email.utils import parseaddr, formataddr
import smtplib

def _format_addr(s):
    name, addr = parseaddr(s)
    return formataddr((Header(name, 'utf-8').encode(), addr))

# 输入email地址和口令
from_addr = input('From:')
password = input('Passwd:')
# 输入收件人地址
to_addr = input('To:')
# 输入SMTP服务器地址
smtp_server = input('SMTP server:')

#msg = MIMEText('hello, send by Python...', 'plain', 'utf-8')
msg = MIMEText('<html><body><h1>Hello</h1>' +
              '<p>send by <a href="http://www.python.org">Python</a>...</p>' +
              '</body></html>', 'html', 'utf-8')
msg['From'] = _format_addr('Python爱好者 <%s>' % from_addr)
msg['To'] = _format_addr('发给自己 <%s>' % to_addr)
msg['Subject'] = Header('Python发送邮件', 'utf-8').encode()

# 通过SMTP发送出去：
server = smtplib.SMTP(smtp_server, 25) # SMTP协议默认端口是25
server.set_debuglevel(1)
server.login(from_addr, password)
server.sendmail(from_addr, [to_addr], msg.as_string())
server.quit()


From:410
Passwd:
To:
SMTP server:


send: 'ehlo lingfangyuan.gialen.com\r\n'


SMTPServerDisconnected: please run connect() first

2. 发送HTML邮件
  1. 发送HTML邮件，在构造MIMEText对象时，把HTML字符串传进去，再把第二个参数由plain变为html就可以了
  2. 发送附件。
    - 构造一个MIMEMultipar对象代表邮件本身，然后往里面加上一个MIMEText作为邮件正文，再继续往里面加上表示附件的MIMEText对象。
  3. 发送图片。
    - 按照发送附件的方式，先把图片作为附件添加进去，任何，在HTML中通过引用src="cid:0"就可以把附件作为图片嵌入。
3. 同时支持HTML和Plain格式。
  1. 利用MIMEMultipar就可以组合一个HTML和Plain，知道subtype是alternative。
4. 只需要在创建SMTP对象后，立刻用starttls()，就创建了安全连接。

In [12]:
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
# 邮件对象:
msg = MIMEMultipart()
msg['From'] = _format_addr('Python爱好者 <%s>' % from_addr)
msg['To'] = _format_addr('管理员 <%s>' % to_addr)
msg['Subject'] = Header('来自SMTP的问候……', 'utf-8').encode()

# 邮件正文是MIMEText
msg.attach(MIMEText('send with file...', 'plain', 'utf-8'))

# 添加附件就是加上一个MIMEBase,从本地读取一个图片
with open('cat.jpg', 'rb') as f:
    # 设置附件的MIME和文件名，这里是jpg类型
    mime = MIMEBase('image', 'jpg', filaname='cat.jpg')
    # 加上必要的头信息
    mime.add_header('Content-Disposition', 'attachment', filename='cat.jpg')
    mime.add_header('Content-ID', '<0>')
    mime.add_header('X-Attachment-Id', '0')
    # 把附件的内容读进来
    mime.set_payload(f.read())
    # 添加到MIMEMultipar
    msg.attach(mime)

构造一个邮件对象就是一个Messag对象，如果构造一个MIMEText对象，就表示一个文本邮件对象，如果构造一个MIMEImage对象，就表示一个作为附件的图片，要把多个对象组合起来，就用MIMEMultipart对象，而MIMEBase可以表示任何对象。它们的继承关系如下：

Message
 - MIMEBase
   - MIMEMultipart
   - MIMENonMultipart
      - MIMEMessage
      - MIMEText
      - MIMEImage